## This jupyter notebook shows how we reimplement the experiment on non-robust features

In [1]:
from __future__ import print_function
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import datasets, transforms
from torch.autograd import Variable

from utils import *
from attack_functions import *
from resnet_cifar import resnet20_cifar
from resnet_cifar import resnet56_cifar
from tqdm import tqdm, trange

import logging
import os
import sys

import torchvision.utils
import matplotlib.pyplot as plt

In [2]:
# Training settings
name = 'cifar10'
batch_size = 128
test_batch_size = 200
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)


In [7]:
model = resnet56_cifar()
model = torch.nn.DataParallel(model)
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

# get dataset
train_loader, test_loader = getData(name=name, train_bs=batch_size, test_bs=test_batch_size)

    Total params: 0.86M
Files already downloaded and verified


In [ ]:
file_path = '../checkpoint/resnet56_cifar_300.pkl'

In [9]:
model.load_state_dict(torch.load(f"{file_path}"))
test(model, test_loader)

Testing
testing_correct:  0.9142 



0.9142

In [11]:
def eval_model(model_tst, loader_tst, dataset_name):

    print("Test the model")

    model_tst.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loader_tst:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_tst(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            #print(total)

        print('Test Accuracy of the model on the ' + dataset_name + ' : {} %'.format(100 * correct / total))
        
    return 100 * correct / total


In [8]:
criterion = nn.CrossEntropyLoss() 

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda')

In [13]:
def imshow(img, title):
    npimg = img.numpy()
    fig = plt.figure(figsize = (10, 30))
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.title(title)
    plt.show()

In [14]:
eval_model(model, test_loader, "original test images")

Test the model
Test Accuracy of the model on the original test images : 91.42 %


91.42

In [15]:
Test_list_log = []
batch_size = 200
learning_rate = 0.01

def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

## Attack part

In [16]:
PGD_attack = PGD_l2(model = model, eps = 10, iters=100, alpha = 2)
adv_data = PGD_attack.save(data_loader=train_loader, file_name="data/cifar_PGD_larger.pt", to_uint8=False, accuracy=True)
#adv_data = PGD_attack.load(file_name="data/cifar_PGD_larger.pt", scale=False)

adv_loader = torch.utils.data.DataLoader(dataset = adv_data, batch_size=batch_size, shuffle=True)

- Save Progress : 0.26 %        
- Accuracy of the model : 0.000000 %- Save Progress : 0.51 %        
- Accuracy of the model : 0.000000 %- Save Progress : 0.77 %        
- Accuracy of the model : 0.000000 %- Save Progress : 1.02 %        
- Accuracy of the model : 0.000000 %- Save Progress : 1.28 %        
- Accuracy of the model : 0.000000 %- Save Progress : 1.53 %        
- Accuracy of the model : 0.000000 %- Save Progress : 1.79 %        
- Accuracy of the model : 0.000000 %- Save Progress : 2.05 %        
- Accuracy of the model : 0.000000 %- Save Progress : 2.30 %        
- Accuracy of the model : 0.000000 %- Save Progress : 2.56 %        
- Accuracy of the model : 0.000000 %- Save Progress : 2.81 %        
- Accuracy of the model : 0.000000 %- Save Progress : 3.07 %        
- Accuracy of the model : 0.000000 %- Save Progress : 3.32 %        
- Accuracy of the model : 0.000000 %- Save Progress : 3.58 %        
- Accuracy of the model : 0.000000 %- Save Progress : 3.84 %        
-

In [ ]:
for i, (images, labels) in enumerate(adv_loader):
    if i < 1 :
        print([np.min(images.data.numpy()), np.max(images.data.numpy())])
        imshow(torchvision.utils.make_grid(images.cpu().data, normalize=True), labels.cpu().numpy()) 
    else:
        break

In [18]:
model_adv = resnet20_cifar()
model_adv.fc = nn.Linear(64,10)
model_adv = model_adv.to(device)
model_adv = nn.DataParallel(model_adv)

In [19]:
optimizer_adv = torch.optim.SGD(model_adv.parameters(), lr=learning_rate, momentum=0.9,
                            weight_decay=0.0005, nesterov=True)
print("Train the confused model")
Loss_list = []
Test_list = []

num_epochs = 100

best_acc = 0
total_step = len(adv_loader)

for epoch in range(num_epochs):

    model_adv.train()
    if epoch > 75:
        lr = 0.0001
    elif epoch > 50:
        lr = 0.001
    else:
        lr = 0.01

    update_lr(optimizer_adv, lr)

    for i, (adv_images, adv_labels) in enumerate(adv_loader):

        # train
        adv_images = adv_images.to(device)
        adv_labels = adv_labels.to(device)

        outputs_adv = model_adv(adv_images)
        loss = criterion(outputs_adv, adv_labels)

        optimizer_adv.zero_grad()
        loss.backward()
        optimizer_adv.step()

        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            Loss_list.append(loss.item())

    if (epoch+1)%1 == 0:

        acc = eval_model(model_adv, test_loader, "original test set")

        Test_list.append(acc)

        if acc > best_acc:
            print('Saving model ' + '..')
            state = {
                'net': model_adv.state_dict(),
                'acc': acc,
                'epoch': epoch,
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/resnet_adv_cifar_larger.pth')
            best_acc = acc

print(Test_list)

Train the confused model
Epoch [1/100], Step [10/250], Loss: 2.3266
Epoch [1/100], Step [20/250], Loss: 2.3003
Epoch [1/100], Step [30/250], Loss: 2.3179
Epoch [1/100], Step [40/250], Loss: 2.3106
Epoch [1/100], Step [50/250], Loss: 2.3075
Epoch [1/100], Step [60/250], Loss: 2.3039
Epoch [1/100], Step [70/250], Loss: 2.2858
Epoch [1/100], Step [80/250], Loss: 2.3037
Epoch [1/100], Step [90/250], Loss: 2.2947
Epoch [1/100], Step [100/250], Loss: 2.2944
Epoch [1/100], Step [110/250], Loss: 2.3014
Epoch [1/100], Step [120/250], Loss: 2.2954
Epoch [1/100], Step [130/250], Loss: 2.3163
Epoch [1/100], Step [140/250], Loss: 2.3245
Epoch [1/100], Step [150/250], Loss: 2.3188
Epoch [1/100], Step [160/250], Loss: 2.3081
Epoch [1/100], Step [170/250], Loss: 2.2968
Epoch [1/100], Step [180/250], Loss: 2.3101
Epoch [1/100], Step [190/250], Loss: 2.3246
Epoch [1/100], Step [200/250], Loss: 2.2962
Epoch [1/100], Step [210/250], Loss: 2.3152
Epoch [1/100], Step [220/250], Loss: 2.2923
Epoch [1/100], S